# Preparação

## Bibliotecas

In [ ]:
# Extração
!pip -q install --upgrade py7zr wget unidecode

# Google CLoud: BigQuery e GCS
!pip install --upgrade -q google-cloud-bigquery google-cloud-storage

In [2]:
# Google Colab e serviços de Cloud
from google.colab import auth
from google.cloud import storage

# Bibliotecas padrão
import os
import io
import time
import shutil
import tempfile
import logging
from datetime import datetime
from typing import Optional, List

# Manipulação de dados
import pandas as pd
import numpy as np

# Extração
import requests
import wget
import ftplib
from py7zr import SevenZipFile
from bs4 import BeautifulSoup
from unidecode import unidecode

## Configurando

In [3]:
# Autenticar
auth.authenticate_user()

In [ ]:
# Inicializa o cliente do GCS
PROJECT_ID = ""
BUCKET_NAME = ''
PATH_GCS = ''

gcs_client = storage.Client(project=PROJECT_ID)
bucket = gcs_client.get_bucket(BUCKET_NAME)

In [5]:
# Outros parâmetros
ftp_base = 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/'

TMP_DIR = '/tmp/colab_caged'
os.makedirs(TMP_DIR, exist_ok=True)

In [6]:
# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
    )

logger = logging.getLogger("caged_etl")

## Funções

In [7]:
def carregar_dados_dieese() -> Optional[pd.DataFrame]:
    """
    Executa o web scraping da página do DIEESE para obter a série histórica
    do Salário Mínimo Nominal e do Salário Mínimo Necessário.
    Retorna um DataFrame ou None em caso de falha.

    A Metodologia do cálculo do SM Necessário pode ser consultada em:
    https://www.dieese.org.br/metodologia/metodologiaCestaBasica2016.pdf
    """
    try:
        logger.info("🚌 Extraindo dados de referência do DIEESE...")

        # Visualizando html da página
        url = "https://www.dieese.org.br/analisecestabasica/salarioMinimo.html"
        response = requests.get(url, timeout=15)
        pagina = BeautifulSoup(response.text, "html.parser")

        # Criando a tabela através dos elementos na página
        linhas = pagina.find_all('tr')
        dados = []
        ano_atual = None

        for linha in linhas:
            # Verifica se é um ano (class="subtitulo")
            if "subtitulo" in linha.get("class", []):
                ano_tag = linha.find("a")
                if ano_tag:
                    ano_atual = ano_tag.get_text().strip()
            # Verifica se é uma linha de dados (exatamente 3 colunas (td))
            elif len(linha.find_all("td")) == 3 and ano_atual is not None:
                colunas = linha.find_all("td")
                mes = colunas[0].get_text(strip=True).replace("\n", "")
                salario_nominal = colunas[1].get_text(strip=True).replace("\n", "")
                salario_necessario = colunas[2].get_text(strip=True).replace("\n", "")


                dados.append({
                    "Ano": ano_atual, "Mês": mes,
                    "Salário Mínimo Nominal": salario_nominal,
                    "Salário Mínimo Necessário": salario_necessario
                })

        if not dados:
            logger.warning("Nenhum dado extraído do site do DIEESE.")
            return None

        sm_dieese = pd.DataFrame(dados)

        # Convertendo informação de data
        meses_para_numeros = {
            'Janeiro': '01', 'Fevereiro': '02', 'Março': '03', 'Abril': '04',
            'Maio': '05', 'Junho': '06', 'Julho': '07', 'Agosto': '08',
            'Setembro': '09', 'Outubro': '10', 'Novembro': '11', 'Dezembro': '12'
        }

        sm_dieese['Data'] = sm_dieese['Ano'].astype(str) + '-' + sm_dieese['Mês']
        sm_dieese['Data'] = sm_dieese['Data'].replace(meses_para_numeros, regex=True)
        sm_dieese['Data'] = pd.to_datetime(sm_dieese['Data'] + '-01', format='%Y-%m-%d')
        sm_dieese.drop(columns=['Ano', 'Mês'], inplace=True)

        # Limpeza das colunas monetárias
        for col in ['Salário Mínimo Nominal', 'Salário Mínimo Necessário']:
            temp_col = (sm_dieese[col]
                        .str.strip()
                        .str.replace("R\$", "", regex=True)
                        .str.replace(r"\.", "", regex=False)
                        .str.replace(",", ".", regex=False)
                        )
            sm_dieese[col] = pd.to_numeric(temp_col, errors='coerce')

        # Renomeia as colunas para facilitar o merge
        sm_dieese.rename(columns={
            'Data': 'competenciamov',
            'Salário Mínimo Nominal': 'salario_minimo_nominal',
            'Salário Mínimo Necessário': 'salario_minimo_necessario'
        }, inplace=True)

        logger.info("✅ Dados do DIEESE extraídos e formatados com sucesso.")
        return sm_dieese[['competenciamov', 'salario_minimo_nominal', 'salario_minimo_necessario']]
    except requests.RequestException as e:
        logger.error(f"Falha na requisição ao site do DIEESE: {e}")
        return None
    except Exception as e:
        logger.error(f"Erro inesperado ao processar dados do DIEESE: {e}")
        return None

In [8]:
def transformar_caged(df: pd.DataFrame, tipo_arquivo: str, df_dieese: Optional[pd.DataFrame]) -> pd.DataFrame:
  """
  Aplica transformações iniciais, conversões de tipo e enriquecimento no DataFrame do CAGED.

  Parâmetros:
        df (pd.DataFrame): DataFrame bruto do Novo CAGED, já lido a partir do TXT.
        tipo_arquivo (str): Tipo do arquivo ('CAGEDMOV', 'CAGEDFOR' ou 'CAGEDEXC').
        df_dieese (pd.DataFrame, opcional): DataFrame com informações mensais do DIEESE
            (salário mínimo nominal e necessário). Pode ser None.

    Retorna:
        pd.DataFrame: DataFrame transformado e com colunas relevantes padronizadas.
  """
  logger.info(f"Iniciando transformação de {len(df)} registros...")

  # 1. Padroniza nomes das colunas para evitar problemas de chave (minúsculas, sem acentos e caracteres especiais)
  df.columns = [unidecode(col).lower().replace(' ', '_').replace('.', '') for col in df.columns]

  # 2. Converte tipos para colunas-chave e numéricas
  df['saldomovimentacao'] = pd.to_numeric(df['saldomovimentacao'], errors='coerce')
  df['competenciamov'] = pd.to_datetime(df['competenciamov'].astype(str) + '01', format='%Y%m%d', errors='coerce')
  df['salario'] = df['salario'].str.replace(',', '.', regex=False)
  df['salario'] = pd.to_numeric(df['salario'], errors='coerce').fillna(0)
  df['idade'] = pd.to_numeric(df['idade'], errors='coerce')
  df['regiao'] = pd.to_numeric(df['regiao'], errors='coerce')

  # 3. Ajusta sinal de admissões e desligamentos no caso de arquivos de exclusão (CAGEDEXC)
  #    - No CAGEDEXC, movimentações são excluídas, então o sinal é invertido.
  if tipo_arquivo == "CAGEDEXC":
      df['admissoes'] = np.where(df['saldomovimentacao'] == 1, -1, 0)
      df['desligamentos'] = np.where(df['saldomovimentacao'] == -1, -1, 0)
      df['saldomovimentacao'] = np.where(df['saldomovimentacao'] == 1, -1, 1)
  else:
      df['admissoes'] = np.where(df['saldomovimentacao'] == 1, 1, 0)
      df['desligamentos'] = np.where(df['saldomovimentacao'] == -1, 1, 0)

  # 4. Integra dados do DIEESE (salário mínimo nominal e necessário) por competência
  if df_dieese is not None:
      df = df.merge(df_dieese, on='competenciamov', how='left')
  else:
      logger.warning("DataFrame do DIEESE não fornecido. Faixas salariais não serão calculadas.")
      df['salario_minimo_nominal'] = np.nan
      df['salario_minimo_necessario'] = np.nan

  # 5. Cria faixas salariais com base no salário mínimo nominal do DIEESE
  sm_nominal = df['salario_minimo_nominal']
  condicoes = [
      (df['salario'] > 0) & (df['salario'] <= 1 * sm_nominal),
       (df['salario'] > 1 * sm_nominal) & (df['salario'] <= 2 * sm_nominal),
        (df['salario'] > 2 * sm_nominal) & (df['salario'] <= 3 * sm_nominal),
         (df['salario'] > 3 * sm_nominal) & (df['salario'] <= 4 * sm_nominal),
          (df['salario'] > 4 * sm_nominal) & (df['salario'] <= 5 * sm_nominal),
           (df['salario'] > 5 * sm_nominal) & (df['salario'] <= 10 * sm_nominal)
  ]
  opcoes = ['Até 1 SM', '1 a 2 SM', '2 a 3 SM', '3 a 4 SM', '4 a 5 SM', '5 a 10 SM']
  df['faixa_salarial'] = np.select(condicoes, opcoes, default='Acima de 10 SM')
  df.loc[(df['salario'] == 0) | (sm_nominal.isna()), 'faixa_salarial'] = 'Não Informado'

  # 6. Cria faixas etárias a partir da idade (pd.cut facilita o binning)
  bins_idade = [9, 14, 17, 24, 29, 39, 49, 64, float('inf')]
  labels_idade = ['10-14 anos', '15-17 anos', '18-24 anos', '25-29 anos', '30-39 anos', '40-49 anos', '50-64 anos', '65+ anos']
  df['faixa_etaria'] = pd.cut(df['idade'], bins=bins_idade, labels=labels_idade, right=True)

  # 7. Seleciona e ordena colunas relevantes para o output
  colunas_finais = [
      'competenciamov', 'uf', 'municipio', 'secao', 'subclasse', 'cbo2002ocupacao',
      'graudeinstrucao', 'idade', 'faixa_etaria', 'horascontratuais', 'racacor', 'sexo', 'salario',
      'faixa_salarial', 'salario_minimo_necessario', 'admissoes', 'desligamentos', 'saldomovimentacao'
  ]
  colunas_existentes = [col for col in colunas_finais if col in df.columns]

  logger.info("✅ Transformação concluída.")
  return df[colunas_existentes]

In [10]:
def pipeline_etl(ano: int, mes: int, df_dieese: Optional[pd.DataFrame], tipo_especifico: Optional[str] = None):
    """
    Executa o pipeline ETL para os microdados do Novo CAGED, do download à carga no Google Cloud Storage.

    O Novo CAGED inicia em janeiro de 2020, e esta função não processa períodos anteriores devido a diferenças metodológicas definidas pelo MTE.

    Tipos de arquivo processados:
        - CAGEDMOV: Movimentações declaradas dentro do prazo (competência igual a AAAAMM).
        - CAGEDFOR: Movimentações declaradas fora do prazo (competência igual a AAAAMM).
        - CAGEDEXC: Movimentações excluídas (competência de exclusão igual a AAAAMM).

    Etapas do processo:
        1. Baixa o arquivo .7z do FTP do MTE.
        2. Extrai o .txt em memória.
        3. Aplica transformações e enriquecimento via `transformar_caged`.
        4. Salva como Parquet e envia para o GCS.

    Args:
        ano (int): Ano do arquivo (>= 2020).
        mes (int): Mês do arquivo (1-12).
        df_dieese (Optional[pd.DataFrame]): DataFrame com informações salariais do DIEESE para integração.
        tipo_especifico (Optional[str], default=None): Um dos valores ["CAGEDMOV", "CAGEDFOR", "CAGEDEXC"].
            Se None, processa todos.

    Raises:
        Exception: Erros de conexão, leitura, ou transformação (com retentativas automáticas).

    Returns:
        None
    """
    # Configurações do Pipeline
    if ano < 2020:
        logger.warning(f"Ano {ano} é anterior ao Novo CAGED. Ignorando...")
        return

    tipos_padrao  = ['CAGEDEXC', 'CAGEDFOR', 'CAGEDMOV']
    tipos_a_processar = [tipo_especifico] if tipo_especifico else tipos_padrao
    mes_str = str(mes).zfill(2)

    servidor_ftp = 'ftp.mtps.gov.br'
    caminho_remoto_base = '/pdet/microdados/NOVO CAGED/'

    # Para caso haja problema com a conexão com o servidor
    numero_de_tentativas = 3

    # Início do Processo com Diretório Temporário
    with tempfile.TemporaryDirectory() as tmpdir:
        for tipo in tipos_a_processar:
            # Loop de retentativas para cada arquivo
            for tentativa in range(numero_de_tentativas):
                try:
                    nome_arquivo_7z = f'{tipo}{ano}{mes_str}.7z'
                    caminho_remoto = f'{caminho_remoto_base}{ano}/{ano}{mes_str}/'
                    caminho_local_7z = os.path.join(tmpdir, nome_arquivo_7z)

                    if tentativa == 0:
                        start_time_str = datetime.now().strftime("%H:%M")
                        print(f"[{start_time_str}] --- Iniciando pipeline para: {nome_arquivo_7z} ---")


                    # DOWNLOAD
                    print("  [1/4] Baixando arquivo via FTP (pode demorar)...")
                    with ftplib.FTP(servidor_ftp, timeout=90) as ftp:
                        ftp.login() # Login anônimo
                        ftp.cwd(caminho_remoto)
                        with open(caminho_local_7z, 'wb') as f:
                            ftp.retrbinary(f'RETR {nome_arquivo_7z}', f.write)
                    print("  ✅ Download concluído.")

                    # VERIFICAÇÃO DE INTEGRIDADE
                    tamanho_bytes = os.path.getsize(caminho_local_7z)
                    with open(caminho_local_7z, 'rb') as f:
                        primeiros_bytes = f.read(6)
                    if tamanho_bytes < 1000 or primeiros_bytes != b"7z\xbc\xaf'\x1c":
                        print(f"  ❌ Arquivo baixado é inválido ou corrompido. Pulando.")
                        continue

                    # EXTRAÇÃO
                    print("  [2/4] Extraindo arquivo .txt...")
                    with SevenZipFile(caminho_local_7z, mode='r') as z:
                        z.extractall(path=tmpdir)
                    print("  ✅ Extração concluída.")

                    # LEITURA PARA DATAFRAME
                    caminho_local_txt = os.path.join(tmpdir, f"{tipo}{ano}{mes_str}.txt")
                    df_raw = pd.read_csv(
                        caminho_local_txt,
                        sep=';',
                        dtype=str,
                        low_memory=False,
                        encoding='utf-8'
                    )

                    # TRANSFORMAÇÃO
                    print(f"  [3/4] Transformando {len(df_raw)} registros...")
                    df_processed = transformar_caged(df_raw, tipo, df_dieese)
                    print("  ✅ Transformação concluída.")

                    # CARGA PARA O CLOUD STORAGE
                    print("  [4/4] Enviando arquivo Parquet para o Cloud Storage...")
                    path_gcs_completo = f"{PATH_GCS}/ano={ano}/mes={mes_str}/{tipo}{ano}{mes_str}.parquet"
                    blob = bucket.blob(path_gcs_completo)
                    blob.upload_from_string(df_processed.to_parquet(index=False))
                    end_time_str = datetime.now().strftime("%H:%M")
                    print(f"[{end_time_str}]   ✅ Carga concluída com sucesso para: {path_gcs_completo}")

                    break

                except ftplib.error_perm:
                    print(f"  ❌ Arquivo não encontrado no servidor. Não haverá novas tentativas.")
                    break
                except Exception as e:
                    print(f"  ⚠️ Tentativa {tentativa + 1} de {numero_de_tentativas} falhou: {e}")
                    if tentativa + 1 < numero_de_tentativas:
                            tempo_espera = 15 * (2 ** tentativa)
                            print(f"  -> Aguardando {tempo_espera} segundos para tentar novamente...")
                            time.sleep(tempo_espera)
                    else:
                            logger.error(f"  ❌ Todas as {numero_de_tentativas} tentativas falharam para {nome_arquivo_7z}. Desistindo.")

            print("  -> Pausando por 10 segundos para não sobrecarregar o servidor...")
            time.sleep(10)

# Execução

## Completa (2020 - Hoje)

Na data em que a extração foi realizada, os arquivos na origem passavam por atualizações - isso acarretou em pastas vazias ou com arquivos corrompidos, gerando os problemas listados abaixo.

| Data       | Período    | Arquivo(s)         | Log                              | Status                                      | Próxima Ação                           |
| ---------- | ---------- | ------------------ | -------------------------------- | ------------------------------------------- | -------------------------------------- |
| 2025-08-12 | 2020/01    | CAGEDEXC, CAGEDFOR | ❌ Não encontrado no servidor     | ✅ Comportamento esperado. Nenhuma ação.     | —                                      |
| 2025-08-12 | 2020/02 | CAGEDEXC           | ❌ Não encontrado no servidor     | ✅ Comportamento esperado. Nenhuma ação.     | —                                      |
| 2025-08-12 | 2020/03 | CAGEDEXC           | ❌ Não encontrado no servidor     | ✅ Comportamento esperado. Nenhuma ação.     | —                                      |
| 2025-08-12 | 2020/03    | CAGEDFOR           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2020/07    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2020/12    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/04    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/05    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/06    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/07    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/08    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/09    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/10    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/11    | CAGEDFOR, CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2021/12    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/01    | CAGEDFOR           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/02    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/03    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/04    | CAGEDFOR, CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/06    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2022/07    | CAGEDMOV           | ❌ Arquivo inválido ou corrompido | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2025/05    | CAGEDEXC, CAGEDFOR, CAGEDMOV           | ❌ A pasta de arquivos na origem estava vazia na data de extração | ⚠️ Pipeline pulou arquivo e seguiu execução | Monitorar e notificar MTE se persistir |
| 2025-08-12 | 2025/07    | CAGEDEXC, CAGEDFOR, CAGEDMOV           | ❌ Não encontrado no servidor     | ✅ Comportamento esperado. Nenhuma ação.     | —                                      |
| 2025-08-12 | 2025/08    | CAGEDEXC, CAGEDFOR, CAGEDMOV           | ❌ Não encontrado no servidor     | ✅ Comportamento esperado. Nenhuma ação.     | —                                      |






In [ ]:
# 1. Carrega a tabela de referência do DIEESE uma única vez para ser usada no loop.
tabela_referencia_dieese = carregar_dados_dieese()

if tabela_referencia_dieese is not None:

    # 2. Define o ano e mês atuais para saber onde parar.
    ano_atual = datetime.now().year
    mes_atual = datetime.now().month

    logger.info(f"O ETL será executado de 01/2020 até {mes_atual:02d}/{ano_atual}.")

    # 3. Loop pelos anos, de 2020 até o ano atual.
    for ano in range(2020, ano_atual + 1):

        # Define até que mês o loop interno deve ir.
        limite_mes = 12 if ano < ano_atual else mes_atual

        for mes in range(1, limite_mes + 1):
            pipeline_etl(
                ano=ano,
                mes=mes,
                df_dieese=tabela_referencia_dieese
            )
else:
    logger.error("ETL não executado: falha ao carregar dados de referência do DIEESE.")

logger.info("🏁 Pipeline de execução completa finalizado. 🏁")

## Precisa
Reprocessamento de arquivos que falharam devido a problemas de conexão/corrupção ou caso deseje extrair arquivos de um período específico.

In [ ]:
# 1. Lista de arquivos
arquivos_para_reprocessar = [
    "CAGEDFOR202003.7z", "CAGEDMOV202007.7z", "CAGEDMOV202012.7z",
    "CAGEDMOV202104.7z", "CAGEDMOV202105.7z", "CAGEDMOV202106.7z",
    "CAGEDMOV202107.7z", "CAGEDMOV202108.7z", "CAGEDMOV202109.7z",
    "CAGEDMOV202110.7z", "CAGEDFOR202111.7z", "CAGEDMOV202111.7z",
    "CAGEDMOV202112.7z", "CAGEDFOR202201.7z", "CAGEDMOV202202.7z",
    "CAGEDMOV202203.7z", "CAGEDFOR202204.7z", "CAGEDMOV202204.7z",
    "CAGEDMOV202206.7z", "CAGEDMOV202207.7z"
]

# 2. Carrega a tabela de referência do DIEESE, se necessário
if 'tabela_referencia_dieese' not in locals() or tabela_referencia_dieese is None:
    tabela_referencia_dieese = carregar_dados_dieese()

if tabela_referencia_dieese is not None:
    logger.info(f"Serão reprocessados {len(arquivos_para_reprocessar)} arquivos.")

    # 3. Loop através da lista de arquivos para reprocessar
    for nome_arquivo in arquivos_para_reprocessar:

        # Extrai o tipo, ano e mês do nome do arquivo
        tipo = nome_arquivo[:8]
        ano = int(nome_arquivo[8:12])
        mes = int(nome_arquivo[12:14])

        # Chama o pipeline passando o tipo_especifico
        pipeline_etl(
            ano=ano,
            mes=mes,
            df_dieese=tabela_referencia_dieese,
            tipo_especifico=tipo
        )
else:
    logger.error("Reprocessamento não executado: falha ao carregar dados de referência do DIEESE.")

logger.info("🏁 Reprocessamento de falhas finalizado. 🏁")